In [2]:
import pandas as pd #Pandas 모듈 불러오기
import numpy as np
import openpyxl
from datetime import datetime
import matplotlib.pyplot as plt
import scipy.stats as stats
from tqdm import tqdm
tqdm.pandas()
plt.rc('font', family='Malgun Gothic')

In [ ]:
data_가맹 = pd.read_csv('../Data/Preprocessed/사용자분석.csv', encoding='utf-8')
data_가맹.head()

In [4]:
#전처리1. 부여군만 필터링
data_가맹_부여 = data_가맹.loc[data_가맹['시']=='부여군']

이상치 제거 프로세스

In [5]:
#사용자코드별 사용금액 합산
grp_out =  data_가맹_부여.groupby(['사용자코드','연령대'])['사용금액'].sum().reset_index()
grp_out.head(5)

,사용자코드,연령대,사용금액
0,김길자0418345175,60대,283720
1,(재)국제도덕협회9999990013,20대,106080
2,(재)부여군지역공동체활성화재단9999999991,20대,1271040
3,21보건소01023832199,10대,1524500
4,ALE RISHI RAM01033729301,30대,256910


In [64]:
#이상치 제거 함수 정의
def outlier(x, target_column):
    q1 = x[target_column].quantile(0.25)
    q3 = x[target_column].quantile(0.75)
    iqr = q3-q1

    x.loc[((q1 - iqr*1.5) < x[target_column]) & (x[target_column] < (q3 + iqr*1.5)), f'이상치_{target_column}'] = '정상'
    return x

In [7]:
#함수 한 행씩 apply 하기
no_outlier_grp = grp_out.groupby('연령대').progress_apply(lambda x : outlier(x, '사용금액')).reset_index(drop = True)

100%|██████████| 7/7 [00:00<00:00, 203.27it/s]


In [8]:
##이상치 제거한 변수 정의
data_가맹_부여_이상치제거 = data_가맹_부여.loc[data_가맹_부여['사용자코드'].isin(no_outlier_grp.loc[no_outlier_grp['이상치_사용금액'] == '정상','사용자코드'])]

In [9]:
#분석월 정의 21년~23년,  1~10월까지만 분석 데이터로 정의
분석동월 = ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월']

In [10]:
#분석월 정의 21년~23년,  1~10월까지만 분석 데이터로 정의
data_가맹_부여_in_out = data_가맹_부여_이상치제거.loc[(data_가맹_부여_이상치제거['연도']=='2021년') | (data_가맹_부여_이상치제거['연도']=='2022년') | (data_가맹_부여_이상치제거['연도']=='2023년')]
data_가맹_부여_in_out = data_가맹_부여_in_out[data_가맹_부여_in_out['월'].isin(분석동월)]

In [12]:
in_out_location = data_가맹_부여_in_out[data_가맹_부여_in_out['읍/면'] == data_가맹_부여_in_out['가맹점 주소']].groupby(['연도','읍/면']).agg({'사용금액':'sum', '사용자코드':'nunique'}).reset_index() #본인지역 사용 금액, 사용자 수
in_out_location.rename(columns={'사용금액':'상주인구 사용금액','사용자코드':'상주인구 사용자 수'},inplace=True)

in_out_location['상주인구 사용횟수'] = data_가맹_부여_in_out[data_가맹_부여_in_out['읍/면'] == data_가맹_부여_in_out['읍/면']].groupby(['연도','읍/면'])['사용자코드'].count().reset_index()['사용자코드']
in_out_location['유입인구 사용금액'] = data_가맹_부여_in_out[data_가맹_부여_in_out['읍/면'] != data_가맹_부여_in_out['가맹점 주소']].groupby(['연도','읍/면'])['사용금액'].sum().reset_index()['사용금액']
in_out_location['유입인구 사용자 수'] = data_가맹_부여_in_out[data_가맹_부여_in_out['읍/면'] != data_가맹_부여_in_out['가맹점 주소']].groupby(['연도','읍/면'])['사용자코드'].nunique().reset_index()['사용자코드']
in_out_location['유입인구 사용횟수'] = data_가맹_부여_in_out[data_가맹_부여_in_out['읍/면'] != data_가맹_부여_in_out['가맹점 주소']].groupby(['연도','읍/면'])['사용자코드'].count().reset_index()['사용자코드']
in_out_location['사용금액 총합계'] = in_out_location['상주인구 사용금액']+in_out_location['유입인구 사용금액']
in_out_location['사용자 수 총합계'] = in_out_location['상주인구 사용자 수']+in_out_location['유입인구 사용자 수']
in_out_location['사용횟수 총합계'] = in_out_location['상주인구 사용횟수']+in_out_location['유입인구 사용횟수']
grp_in_out = in_out_location.groupby('연도')[in_out_location.columns.difference(['연도','읍/면'])].sum().reset_index()
#사용금액 비율
grp_in_out['상주인구 사용금액 비율'] = round(grp_in_out['상주인구 사용금액']/grp_in_out['사용금액 총합계']*100,1)
grp_in_out['유입인구 사용금액 비율'] = round(grp_in_out['유입인구 사용금액']/grp_in_out['사용금액 총합계']*100,1)

#사용자 수 비율
grp_in_out['상주인구 사용자 수 비율'] = round(grp_in_out['상주인구 사용자 수']/grp_in_out['사용자 수 총합계']*100,1)
grp_in_out['유입인구 사용자 수 비율'] = round(grp_in_out['유입인구 사용자 수']/grp_in_out['사용자 수 총합계']*100,1)

#사용횟수 비율
grp_in_out['상주인구 사용횟수 비율'] = round(grp_in_out['상주인구 사용횟수']/grp_in_out['사용횟수 총합계']*100,1)
grp_in_out['유입인구 사용횟수 비율'] = round(grp_in_out['유입인구 사용횟수']/grp_in_out['사용횟수 총합계']*100,1)
grp_in_out[['연도','상주인구 사용금액', '상주인구 사용자 수', '상주인구 사용횟수', '유입인구 사용금액', '유입인구 사용자 수', '유입인구 사용횟수']]

,연도,상주인구 사용금액,상주인구 사용자 수,상주인구 사용횟수,유입인구 사용금액,유입인구 사용자 수,유입인구 사용횟수
0,2021년,33426790534,37297,1701512,24448592071,32300,678384
1,2022년,31229851810,25477,1671715,22677995821,22595,662776
2,2023년,25187492701,25803,1500190,19987392004,23411,623790


In [ ]:
#거주지 내/외 사용 구분 변수 정의
data_가맹_부여_in_out.loc[data_가맹_부여_in_out['읍/면'] == data_가맹_부여_in_out['가맹점 주소'],'거주지역 내/외 사용구분'] = '거주지역 내 사용'
data_가맹_부여_in_out.loc[data_가맹_부여_in_out['읍/면'] != data_가맹_부여_in_out['가맹점 주소'],'거주지역 내/외 사용구분'] = '거주지역 외 사용'
data_가맹_부여_in_out.head(3)

In [26]:
#충전 데이터와 합치기 위해 기준연월 변수 생성
data_가맹_부여_in_out['기준연월'] = pd.to_datetime(data_가맹_부여_in_out['날짜'], format='%Y%m%d').dt.strftime('%Y/%m')

In [27]:
df_in_out_grp = data_가맹_부여_in_out.groupby(['사용자코드','기준연월','연령대','거주지역 내/외 사용구분','읍/면','가맹점 주소','대분류','소분류'])['사용금액'].sum().reset_index()

충전데이터 구분에 따른 지역내/외 사용 정도 분석

In [17]:
ch_현황_df = pd.read_csv('../Data/Preprocessed/사용자충전데이터.csv', encoding='cp949')
ch_현황_df.head()

,구분,충전금액,발행(정책+민간),인센티브적립,인센티브사용,기준연월,사용자코드
0,일반 사용자,0.0,0.0,0.0,0.0,2020/11,김진선01029019263
1,일반 사용자,0.0,0.0,776277.0,0.0,2020/11,박대웅01042416386
2,일반 사용자,0.0,0.0,72759.0,279941.0,2020/11,유금예01024888183
3,일반 사용자,0.0,0.0,2010.0,0.0,2020/11,이창희01085080732
4,일반 사용자,1000000.0,0.0,165831.0,0.0,2020/11,이광호01056371015


충전별 구분

In [21]:
#자가충전만 한 사용자
ch_현황_df.loc[(ch_현황_df['충전금액']>0) & (ch_현황_df['발행(정책+민간)']<=0),['충전 구분']] = '자가충전만'
#정책+민간 발행만 받은 사용자
ch_현황_df.loc[(ch_현황_df['충전금액']<=0) & (ch_현황_df['발행(정책+민간)']>0),['충전 구분']] = '발행만'
#정책+민간 발행받고 자가충전을 한 사용자
ch_현황_df.loc[(ch_현황_df['충전금액']>0) & (ch_현황_df['발행(정책+민간)']>0),['충전 구분']] = '자가+발행'
#나머지 알수 없음

In [ ]:
ch_현황_df.groupby(['기준연월','사용자코드','충전 구분']).agg({'충전금액':'sum','발행(정책+민간)':'sum'}).reset_index()

In [23]:
ch_data_df = ch_현황_df.groupby(['기준연월','사용자코드','충전 구분']).agg({'충전금액':'sum','발행(정책+민간)':'sum'}).reset_index()

In [29]:
#충전데이터와 사용 데이터 합치기
df_in_out = pd.merge(df_in_out_grp, ch_data_df,how='left',on=['기준연월','사용자코드'])

In [36]:
df_in_out['충전 구분'].unique()

array(['발행만', nan, '자가충전만', '자가+발행'], dtype=object)

In [ ]:
#연도, 월 변수 생성
df_in_out['연도'] = pd.to_datetime(df_in_out['기준연월'],format='%Y/%m').dt.strftime('%Y')
df_in_out['월'] = pd.to_datetime(df_in_out['기준연월'],format='%Y/%m').dt.strftime('%m')
df_in_out

In [47]:
df_in_out_2021 = df_in_out.loc[df_in_out['연도']=='2021']
df_in_out_2022 = df_in_out.loc[df_in_out['연도']=='2022']
df_in_out_2023 = df_in_out.loc[df_in_out['연도']=='2023']

In [48]:
df_in_out_2021.to_excel('../Result/700. 지역균형발전 기여 분석/유입인구 상주인구_2021년.xlsx',index=False)
df_in_out_2022.to_excel('../Result/700. 지역균형발전 기여 분석/유입인구 상주인구_2022년.xlsx',index=False)
df_in_out_2023.to_excel('../Result/700. 지역균형발전 기여 분석/유입인구 상주인구_2023년.xlsx',index=False)

확인 해보니 발행만 받은 사용자들의 지역외 사용 비중이 가장 높게 형성되어 발행만 받은 데이터만 추출하여 현황분석 수행

발행만 받은 데이터 추출

In [56]:
df_in_out_new = df_in_out.loc[(df_in_out['충전 구분']=='발행만') & ((df_in_out['연도']=='2022') | (df_in_out['연도']=='2023'))]

In [58]:
df_in_out_new = df_in_out_new.loc[df_in_out_new['대분류']!='기타']

In [59]:
분석행정동 = ['부여읍','규암면','홍산면']

In [ ]:
df_in_out_new.loc[(~df_in_out_new['읍/면'].isin(분석행정동)),'읍/면'] = '기타'
df_in_out_new.loc[(~df_in_out_new['가맹점 주소'].isin(분석행정동)),'가맹점 주소'] = '기타'
df_in_out_new

In [63]:
df_in_out_new.to_excel('../Result/700. 지역균형발전 기여 분석/2.1 지역균형발전기여(발행만).xlsx',index=False)